# Airline Reliability Clustering Analysis

This notebook replicates the k-means clustering analysis from `airline_clustering.py` with interactive visualizations and cluster exploration.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)


## 1. Load and Clean Data


In [ ]:
# Load data
df = pd.read_csv('./T_ONTIME_REPORTING.csv', low_memory=False)
print(f"Loaded {len(df):,} flight records")
print(f"Columns: {df.columns.tolist()}")

# Map column names to standard format (handle variations)
column_mapping = {}

# Reporting airline
for col in ['Reporting_Airline', 'OP_UNIQUE_CARRIER', 'OP_CARRIER']:
    if col in df.columns:
        column_mapping[col] = 'Reporting_Airline'
        break

# Delay columns
for std_name, variations in [
    ('DepDelay', ['DepDelay', 'DEP_DELAY', 'DEPARTURE_DELAY']),
    ('ArrDelay', ['ArrDelay', 'ARR_DELAY', 'ARRIVAL_DELAY']),
    ('DepDel15', ['DepDel15', 'DEP_DEL15', 'DEP_DELAY_15']),
    ('ArrDel15', ['ArrDel15', 'ARR_DEL15', 'ARR_DELAY_15']),
    ('Cancelled', ['Cancelled', 'CANCELLED', 'CANCELED']),
    ('Diverted', ['Diverted', 'DIVERTED'])
]:
    for var in variations:
        if var in df.columns:
            column_mapping[var] = std_name
            break

# Rename columns
df = df.rename(columns=column_mapping)
print(f"Renamed columns to standard format")


In [ ]:
# Clean data
numeric_cols = ['DepDelay', 'ArrDelay', 'DepDel15', 'ArrDel15', 'Cancelled', 'Diverted']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Clip extreme delays
df['DepDelay'] = df['DepDelay'].clip(-30, 180)
df['ArrDelay'] = df['ArrDelay'].clip(-30, 180)

# Drop missing values
df_clean = df.dropna(subset=numeric_cols + ['Reporting_Airline'])
print(f"Retained {len(df_clean):,} clean flight records")


## 2. Engineer Airline-Level Features


In [ ]:
# Group by airline
min_flights = 2000
airline_stats = df_clean.groupby('Reporting_Airline').agg({
    'DepDelay': 'mean',
    'ArrDelay': 'mean',
    'DepDel15': 'mean',
    'ArrDel15': 'mean',
    'Cancelled': 'mean',
    'Diverted': 'mean',
    'Reporting_Airline': 'count'
}).rename(columns={
    'DepDelay': 'mean_dep_delay',
    'ArrDelay': 'mean_arr_delay',
    'DepDel15': 'pct_dep_delayed_15',
    'ArrDel15': 'pct_arr_delayed_15',
    'Cancelled': 'cancel_rate',
    'Diverted': 'divert_rate',
    'Reporting_Airline': 'flight_count'
})

# Filter by minimum flights
airline_stats = airline_stats[airline_stats['flight_count'] >= min_flights]
print(f"Retained {len(airline_stats)} airlines with ≥{min_flights} flights")

feature_cols = ['mean_dep_delay', 'mean_arr_delay', 'pct_dep_delayed_15',
                'pct_arr_delayed_15', 'cancel_rate', 'divert_rate']

airline_stats.head(10)


## 3. Scale Features and Find Optimal k


In [ ]:
# Scale features
X = airline_stats[feature_cols].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Test k values
k_range = range(3, 9)
results = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, n_init=25, max_iter=500, 
                   random_state=42, init='k-means++')
    labels = kmeans.fit_predict(X_scaled)
    
    inertia = kmeans.inertia_
    silhouette = silhouette_score(X_scaled, labels)
    
    results.append({'k': k, 'inertia': inertia, 'silhouette': silhouette})
    print(f"k={k}: inertia={inertia:.2f}, silhouette={silhouette:.4f}")

# Find best k
best_result = max(results, key=lambda x: x['silhouette'])
best_k = best_result['k']
print(f"\nBest k={best_k} (silhouette={best_result['silhouette']:.4f})")


## 4. Visualizations


In [ ]:
# Elbow plot
k_vals = [r['k'] for r in results]
inertias = [r['inertia'] for r in results]

plt.figure(figsize=(8, 6))
plt.plot(k_vals, inertias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Number of Clusters (k)', fontsize=12)
plt.ylabel('Inertia (Within-Cluster Sum of Squares)', fontsize=12)
plt.title('Elbow Method for Optimal k', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Silhouette plot
silhouettes = [r['silhouette'] for r in results]

plt.figure(figsize=(8, 6))
plt.plot(k_vals, silhouettes, 'go-', linewidth=2, markersize=8)
plt.xlabel('Number of Clusters (k)', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.title('Silhouette Score vs Number of Clusters', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Fit final model
kmeans = KMeans(n_clusters=best_k, n_init=25, max_iter=500,
               random_state=42, init='k-means++')
labels = kmeans.fit_predict(X_scaled)
centroids_scaled = kmeans.cluster_centers_

# PCA projection
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)
centroids_pca = pca.transform(centroids_scaled)

# Plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, 
                     cmap='viridis', s=50, alpha=0.6, edgecolors='k', linewidth=0.5)
plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], 
           c='red', marker='x', s=200, linewidths=3, label='Centroids')

plt.xlabel(f'First Principal Component ({pca.explained_variance_ratio_[0]:.1%} variance)', 
           fontsize=12)
plt.ylabel(f'Second Principal Component ({pca.explained_variance_ratio_[1]:.1%} variance)', 
           fontsize=12)
plt.title('Airline Clusters (2D PCA Projection)', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 5. Cluster Summary


In [ ]:
# Create summary table
summary = airline_stats.copy()
summary['cluster'] = labels

# Summary by cluster
print("Airlines per Cluster:")
for cluster_id in sorted(summary['cluster'].unique()):
    cluster_airlines = summary[summary['cluster'] == cluster_id]
    print(f"\nCluster {cluster_id} ({len(cluster_airlines)} airlines):")
    print(f"  Airlines: {', '.join(cluster_airlines.index.tolist())}")
    print(f"  Total flights: {cluster_airlines['flight_count'].sum():,}")


## 6. Cluster Profiles


In [ ]:
# Get centroids in original scale
centroids_original = scaler.inverse_transform(centroids_scaled)
centroids_df = pd.DataFrame(
    centroids_original,
    columns=feature_cols,
    index=[f'Cluster_{i}' for i in range(best_k)]
)

print("Cluster Centroids (Original Units):")
centroids_df


In [ ]:
# Print cluster profiles
print("\n" + "="*60)
print("CLUSTER PROFILES")
print("="*60)

for cluster_id in range(best_k):
    cluster_airlines = summary[summary['cluster'] == cluster_id]
    mean_features = cluster_airlines[feature_cols].mean()
    
    # Determine profile characteristics
    avg_delay = (mean_features['mean_dep_delay'] + mean_features['mean_arr_delay']) / 2
    avg_delay_pct = (mean_features['pct_dep_delayed_15'] + mean_features['pct_arr_delayed_15']) / 2
    
    if avg_delay < 10 and avg_delay_pct < 0.15:
        profile = "Low delays, low cancel/divert"
    elif avg_delay > 20 or avg_delay_pct > 0.25:
        profile = "High delays, disruption-prone"
    else:
        profile = "Moderate reliability"
    
    print(f"\nCluster {cluster_id}: {profile}")
    print(f"  Airlines: {', '.join(cluster_airlines.index.tolist())}")
    print(f"  Mean dep delay: {mean_features['mean_dep_delay']:.2f} min")
    print(f"  Mean arr delay: {mean_features['mean_arr_delay']:.2f} min")
    print(f"  % Dep delayed ≥15min: {mean_features['pct_dep_delayed_15']:.1%}")
    print(f"  % Arr delayed ≥15min: {mean_features['pct_arr_delayed_15']:.1%}")
    print(f"  Cancel rate: {mean_features['cancel_rate']:.2%}")
    print(f"  Divert rate: {mean_features['divert_rate']:.2%}")
